### VTD Version
Use VTDs instead of precints!

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.spatial import distance_matrix
from shapely.geometry import Polygon
import shapely.speedups


In [2]:
DRIVE_PATH = "/Users/hanlin/Google Drive/CS+/Data/"

def full_path(filename):
    return DRIVE_PATH + filename

In [3]:
precincts = gpd.read_file(full_path("NCabs_VTD"))

In [4]:
precincts.rename(columns={"County":"COUNTY_FIPS", 'VTD_Key':'loc_prec', 'PL10AA_TOT':'total_pop',
                      'PL10VA_TOT':'total_18+', 'EL16G_PR_D':'total_dem', 'EL16G_PR_R':'total_rep',
                      'EL16G_PR_L':'total_lib', 'EL16G_PR_W':'writein', 'EL16G_PR_T':'total_vote',
                      'HVAP':'hispanic', 'WVAP':'white', 'BVAP':'african_am', 'AMINVAP':'am_indian_',
                      'ASIANVAP':'asian', 'NHPIVAP':'hawaii/pac', 'OTHERVAP':'other_race', 
                       '2MOREVAP':'2+races'}, inplace=True)

In [5]:
#THIS IS CRITICAL! NEIGHBORS must be SPLIT!
precincts["neighbors"] = precincts.my_neighbo.str.split(", ")

In [6]:
shapely.speedups.enable()

### Algorithm Functions

In [7]:
n = 9535483

def divide_and_conquer(remaining_districts, district_to_divide, max_function, dist_shape):
    print("divide and conquer")
    if remaining_districts ==1:
        return [district_to_divide]
    
    k = remaining_districts # n= total popoulation, k= remaining_dists in their code
    
    if k%2 ==1:
        rem_district_calls1 = 1
        rem_district_calls2 = k-1
        L = district_to_divide.total_pop.sum()/k
    else:
        rem_district_calls1 = k/2
        rem_district_calls2 = k/2
        L = district_to_divide.total_pop.sum()/2
        
    data, left_shape, right_shape = redistrict_two_districts(district_to_divide, L, max_function, False, dist_shape)
    left_district = data.loc[data.district == 0]
    right_district = data.loc[data.district == 1]
    
    left_district_list = divide_and_conquer(rem_district_calls1, left_district, max_function, left_shape)
    right_district_list = divide_and_conquer(rem_district_calls2, right_district, max_function, right_shape)
    
    all_districts_list = left_district_list + right_district_list  #just naming in case '+' concatenation trips anything up and needs to be changed
    return all_districts_list

In [8]:
def population_swapping(data, L, trying_again, dist0_shape, dist1_shape):
    print("population swapping with", L)
    last_deviation = 100000000000
    #Use loc to guarantee selection of the "0" district
    dist0_pop = data.loc[:, ["district", "total_pop"]].groupby("district").sum().loc[0, "total_pop"]
    dist1_pop = data.loc[:, ["district", "total_pop"]].groupby("district").sum().loc[1, "total_pop"]
    #Their swapping algo is implemented very slowly, I can speed it up so it's essentially always on "maxoptimize"
    while (abs(dist0_pop - L) > 1)  & (abs(dist0_pop - L) < last_deviation):
        last_deviation = abs(dist0_pop - L)
        #Calculate more deviated district
        if abs(dist0_pop - L) > abs(dist1_pop - L):
            more_dev_district = 0
            less_dev_district = 1
            less_dev_pop = dist1_pop
            more_dev_shape = dist0_shape
            less_dev_shape = dist1_shape
        else:
            more_dev_district = 1
            less_dev_district = 0
            less_dev_pop = dist0_pop
            more_dev_shape = dist1_shape
            less_dev_shape = dist0_shape 
            
        #Calculate neighbors of the less deviated district, and add into data
        dist_comp = data.loc[data.district==less_dev_district, "loc_prec"].tolist()
        neighbors = set()
        neighbor_list = data.loc[data.district==less_dev_district, "neighbors"].tolist()
        for one_neighbor_list in neighbor_list:
            for item2 in one_neighbor_list:
                neighbors.add(item2)
        neighbors = neighbors.difference(dist_comp)
        data.loc[data.loc_prec.isin(neighbors), "is_neighbor"] = True
        
        #Reset swappable column
        data["swappable"] = 0 
        
        #Retrieve swappable precincts
        swappable = data.loc[(data.district==less_dev_district) & (data.is_neighbor==True)]
        
        #Sort to pick precinct that minimizes population deviation the most
        swappable["hyp_pop"] = swappable.total_pop + less_dev_pop
        swappable["hyp_dev"] = abs(swappable.hyp_pop - L)
        swappable.sort_values(by="hyp_dev", ascending=True, inplace=True )
        
        #Iterate through, picking the precinct that maintains contiguity
        for index, row in swappable.iterrows():
            new_more_dev = more_dev_shape.difference(row.geometry)
            if not is_contiguous(new_more_dev):
                continue
            new_less_dev = less_dev_shape.union(row.geometry)
            if not is_contiguous(new_less_dev):
                continue
            #Swap from more dev district to less dev district
            data.loc[data.loc_prec==row.loc_prec, "district"] = less_dev_district
            if less_dev_district == 0:
                dist0_shape = new_less_dev
                dist1_shape = new_more_dev
            else:
                dist0_shape = new_more_dev
                dist1_shape = new_less_dev
            break
                
        dist0_pop = data.loc[:, ["district", "total_pop"]].groupby("district").sum().loc[0, "total_pop"]
        dist1_pop = data.loc[:, ["district", "total_pop"]].groupby("district").sum().loc[1, "total_pop"]
    
    return data, dist0_shape, dist1_shape
            
def is_contiguous(new_shape):
    if (str(type(new_shape)) == "<class 'shapely.geometry.polygon.Polygon'>"):
        if len(new_shape.interiors) == 0:
            return True
    return False

In [9]:
def plotter(shape):
    gpd.GeoSeries(shape).plot()
    plt.show()
    
def plotter2(shape1, shape2):
    gpd.GeoSeries(shape1).plot()
    gpd.GeoSeries(shape2).plot()
    plt.show()
    
#Works on any precinct-level data that has distinct districts via a "district" column
def calc_eff_gap(data):
    rep_wasted = []                          
    dem_wasted = []
    votes = []
    #Groups precincts into districts
    data = data.groupby("district").sum()
    #For each district, add to wasted_dem votes and wasted_rep votes
    for index, row in data.iterrows():
        votes.append(row.total_vote)
        if row.total_dem < row.total_rep:
            dem_wasted.append(row.total_dem)  
            rep_wasted.append(row.total_rep - (row.total_vote/2))
        else:
            rep_wasted.append(row.total_rep)
            dem_wasted.append(row.total_dem - (row.total_vote/2))
    #Calculate a statewide net efficiency gap
    total_rep_wasted = sum(rep_wasted)
    total_dem_wasted = sum(dem_wasted)
    total_vote = sum(votes)
    return abs(total_rep_wasted - total_dem_wasted)/total_vote
    

    
def redistrict_two_districts(data, L, max_function, trying_again, dist_shape):
    print("Redistricting with " + str(L) + "= L")
    #Prefer not to use unary_union if there's a better option out there, like if it was passed in
    envelope = np.asarray(dist_shape.envelope.exterior.coords)
    envelope = envelope[0:4] #The envelope returns a rectangle with a duplicate 5th entry, for some reason
    dist_poss = []
    for corner in envelope:
        twodist = modified_voronoi(corner, L, data)
        twodist, dist1_shape, dist2_shape = contiguity_swapping(twodist, dist_shape)
        #plotter2(dist1_shape, dist2_shape)
        
        twodist, dist1_shape, dist2_shape = population_swapping(twodist, L, trying_again, dist1_shape, dist2_shape)        
        dist1_schwartz = ((2 * math.pi)*((dist1_shape.area/math.pi)**0.5))/dist1_shape.length
        dist2_schwartz = ((2 * math.pi)*((dist2_shape.area/math.pi)**0.5))/dist2_shape.length
        avg_mod_schwartz = (dist1_schwartz + dist2_schwartz)/2
        
        dist0_pop = twodist.loc[:, ["district", "total_pop"]].groupby("district").sum().loc[0, "total_pop"]
        dist1_pop = twodist.loc[:, ["district", "total_pop"]].groupby("district").sum().loc[1, "total_pop"]
        
        dist0_pop_dev = abs(dist0_pop - L)/L
        dist1_pop_dev = abs(dist1_pop - L)/L
        max_pop_dev = max(dist0_pop_dev, dist1_pop_dev)
        
        eff_gap = calc_eff_gap(twodist)
        
        dist_poss.append({"data":twodist, "dist1_shape":dist1_shape, "dist2_shape":dist2_shape, 
                         "avg_mod_schwartz":avg_mod_schwartz, "pop_dev":max_pop_dev, "eff_gap":eff_gap})
    #Technically we'd have the MinPop clause trigger, but I've sped up the min pop algorithm so it's not nec.
    data, dist1_shape, dist2_shape = choose_best(dist_poss, max_function)
    return data, dist1_shape, dist2_shape

In [10]:
def choose_best(dist_poss, max_function):
    print("choose best")
    if max_function == "min_pop":
        ret = min(dist_poss, key=lambda d: d['pop_dev'])
        return ret["data"], ret["dist1_shape"], ret["dist2_shape"] #key argument specifies what to compare on
    if max_function == "max_compact":
        ret = max(dist_poss, key=lambda d: d['avg_mod_schwartz'])
        return ret["data"], ret["dist1_shape"], ret["dist2_shape"] 
    if max_function == "valid_compact":
        valid_dist_poss = [x for x in dist_poss if x["pop_dev"] <= 0.005]
        if len(valid_dist_poss) != 0:
            ret = max(valid_dist_poss, key=lambda d: d['avg_mod_schwartz'])
            return ret["data"], ret["dist1_shape"], ret["dist2_shape"]
        else:
            ret = max(dist_poss, key=lambda d: d['avg_mod_schwartz'])
            return ret["data"], ret["dist1_shape"], ret["dist2_shape"]
    if max_function == "valid_effgap":
        valid_dist_poss = [x for x in dist_poss if x["pop_dev"] <= 0.005]
        print(len(valid_dist_poss), len(dist_poss))
        if len(valid_dist_poss) != 0:
            ret = min(valid_dist_poss, key=lambda d: d['eff_gap'])
            return ret["data"], ret["dist1_shape"], ret["dist2_shape"]
        else:
            ret = min(dist_poss, key=lambda d: d['eff_gap'])
            return ret["data"], ret["dist1_shape"], ret["dist2_shape"]
    if max_function == "min_effgap":
        ret = min(dist_poss, key=lambda d: d['eff_gap'])
        return ret["data"], ret["dist1_shape"], ret["dist2_shape"]
    if max_function == "ball":
        #Find the districting which minimizes the maximal"cracked metric" 
        #of any neighboring precinct for a districting
        min_pct_suppressed = 1
        min_idx = 0
        for idx, poss in enumerate(dist_poss):
            data = poss['data']
            neighbors = data.loc[data.is_neighbor==True]
            curr_pct = neighbors.pct_voters_suppressed.max()
            if curr_pct < min_pct_suppressed:
                min_pct_suppressed = curr_pct
                min_idx = idx
        ret = dist_poss[min_idx]
        return ret["data"], ret["dist1_shape"], ret["dist2_shape"]

In [11]:
def modified_voronoi(site, L, district_to_divide):
    print(site, "modified voronoi")
    district_to_divide["centroid"] = district_to_divide.geometry.centroid
    def coord_lister(geom):
        coords = list(geom.coords)
        return (coords)

    coordinates = district_to_divide.centroid.apply(coord_lister)
    coordinates = coordinates.tolist()
    coordinates = [coordinates[i][0] for i in range(len(coordinates))]    
    df = pd.DataFrame(distance_matrix([site], coordinates)).T
    df.rename({0:"distance"}, axis =1, inplace=True)
    #print("LENGTH coord, ", df.shape, district_to_divide.shape)
    district_to_divide.reset_index(inplace=True, drop=True)
    concated = pd.concat([district_to_divide, df], axis=1)
    concated.sort_values(by="distance", ascending=True, inplace=True)
    concated["cum_sum"] = concated.total_pop.cumsum()
    concated.loc[concated.cum_sum <= L, "district"] = 0
    concated.loc[concated.cum_sum > L, "district"] = 1
    concated.drop(columns=["distance", "cum_sum"], inplace=True)
    return concated

In [12]:
def contiguity_swapping(data, dist_shape):
    print("Contiguity swapping")
    dist0_shape = data[data.district==0].unary_union
    dist1_shape = dist_shape.difference(dist0_shape)
    #plotter2(dist0_shape, dist1_shape)
    print("Finished unary union, now determining noncontiguous components")
    #We only care about MultiPolygon case, if it has non-contiguous components
    """
    if dist0_shape.geom_type == "MultiPolygon": 
        max_comp = None
        max_comp_len = 0
        for component in dist0_shape:
            if len(component.interiors) > max_comp_len:
                max_comp = component
                max_comp_len = len(component.interiors)
        dist1_shape = dist1_shape.union((dist0_shape.difference(max_comp)))
        dist0_shape = max_comp
    if dist1_shape.geom_type == "MultiPolygon":
        max_comp = None
        max_comp_len = 0
        for component in dist1_shape:
            if len(component.interiors) > max_comp_len:
                max_comp = component
                max_comp_len = len(component.interiors)
        dist0_shape = dist0_shape.union((dist1_shape.difference(max_comp)))
        dist1_shape = max_comp
        """
    if dist0_shape.geom_type == 'MultiPolygon':
        max_comp = max(dist0_shape, key=lambda a: a.area)
        dist1_shape = dist1_shape.union(dist0_shape.difference(max_comp))
        dist0_shape = max_comp
    if dist1_shape.geom_type == 'MultiPolygon':
        max_comp = max(dist1_shape, key=lambda a: a.area)
        dist0_shape = dist0_shape.union(dist1_shape.difference(max_comp))
        dist1_shape = max_comp
    
    print("Relabeling dataframe with correct numbers")
    #Relabel the DataFrame with the correct district numbers, using centroid technique
    #Centroids aren't actually much faster than area of intersection. Will have to investigate which to use.
    
    
    data["district"] = 1
    data.loc[data.centroid.within(dist0_shape), "district"] = 0
    #plotter2(dist0_shape, dist1_shape)
        
    return data, dist0_shape, dist1_shape

### Visualization and Processing Funcs

In [13]:
def relabel_and_combine(final_lists):
    i = 1
    for df in final_lists:
        df["district"] = i
        i+= 1
    finaldf = pd.concat(final_lists)
    return finaldf
    
def plot_dist_partisaness(finaldf):
    #Expensive operation
    districts = finaldf.dissolve(by="district", aggfunc="sum")
    districts["rep_prop"] = (districts["total_rep"]/districts["total_vote"]).round(3)
    districts['coords'] = districts['geometry'].apply(lambda x: x.representative_point().coords[:])
    districts['coords'] = [coords[0] for coords in districts['coords']]
    districts.plot(figsize=(18, 16),
               column="rep_prop", cmap="coolwarm", legend=True, vmin=0, vmax=1, edgecolor="black")
    plt.title("Rep Vote Percentage Per District")
    for idx, row in districts.iterrows():
        plt.annotate(s=row['rep_prop'], xy=row['coords'],
                     horizontalalignment='center')
    print(calc_eff_gap(finaldf))

### Testing Various Maximization Functions

In [14]:
#Run this only once
dist_shape = precincts.unary_union

In [15]:
#Main Call
final_lists = divide_and_conquer(13, precincts, "valid_compact", dist_shape)

divide and conquer
Redistricting with 733498.6923076923= L
[123998.5227    822.5377] modified voronoi
Contiguity swapping
Finished unary union, now determining noncontiguous components
Relabeling dataframe with correct numbers
population swapping with 733498.6923076923
[9.35803799e+05 8.22537700e+02] modified voronoi


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Contiguity swapping
Finished unary union, now determining noncontiguous components
Relabeling dataframe with correct numbers
population swapping with 733498.6923076923
[935803.7987 318099.5218] modified voronoi


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Contiguity swapping
Finished unary union, now determining noncontiguous components
Relabeling dataframe with correct numbers
population swapping with 733498.6923076923
[123998.5227 318099.5218] modified voronoi


...

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

Finished unary union, now determining noncontiguous components
Relabeling dataframe with correct numbers
population swapping with 736030.5
[632478.3815 315360.0149] modified voronoi
Contiguity swapping


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Finished unary union, now determining noncontiguous components
Relabeling dataframe with correct numbers
population swapping with 736030.5
choose best
divide and conquer
divide and conquer


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [54]:
#Merely to save a shapefile of the valid_compact data, not creating the VTD-district pairs
'''finaldf = relabel_and_combine(final_lists)
finaldf.neighbors = finaldf.neighbors.to_string()
finaldf.drop(columns = ["centroid"], inplace=True)
finaldf.to_file('valid_compact.shp')'''


In [18]:
finaldf = relabel_and_combine(final_lists)
#plot_dist_partisaness(finaldf)

In [19]:
#Very important to not save header or index to match other inputs
finaldf[['VTD_num', 'district']].to_csv("acr_VTD.txt", sep='\t', header=False, index=False)